In [ ]:
from modules import *
%matplotlib widget

### 1. Load dataset

In [ ]:
datapicker = create_datapicker()
display(datapicker)

### 2. Explore Dataset Maps

In [ ]:
exploremap(datapicker.geodata, column='tk_airquality_iaqindex_value')

### 3. Explore Dataset Streams

In [ ]:
plot_traces({
    'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
    'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
    'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
    'altitude': datapicker.dataset.georeference.elevation,
    'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
    'ecg': datapicker.dataset.streams.BioData.ECG.data,
    'e4': datapicker.dataset.streams.Empatica.data.E4_Gsr.Value,
})

# plot_traces({
#     # 'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
#     # 'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
#     # 'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
#     # 'altitude': datapicker.dataset.georeference.elevation,
#     # 'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
#     # 'e4': datapicker.dataset.streams.E4.Gsr,
# })

### 4. Export Dataset to OGC API

In [ ]:
record = DatasetRecord(datapicker.dataset, datapicker.geodata, properties=RecordProperties(
    title='<City> Outdoor Walk: <CityRegion> Subject <ID>',
    description='Outdoor walk data collection',
    license='CC BY-NC 4.0',
    tool='Bonsai',
    keywords=['<City>', 'Outdoor', 'Walk', 'Microclimate', 'Biosignals'],
    contacts=[
        Contact(
            name='Your Name',
            institution='Your Institution',
            email='youremail@yourdomain.com'
        )
    ],
    themes=[]
))

In [ ]:
rpath = Path(record.id)
export_geoframe_to_geojson(datapicker.geodata, rpath.with_suffix('.geojson'))
with open(rpath.with_suffix('.json'), 'w') as f:
    f.write(record.to_json())

### 5. Export gaze data 

In [ ]:
import pandas as pd
gaze_timestamps = datapicker.dataset.streams.PupilLabs.Counter.Gaze.data
#gaze_timestamps.reset_index(inplace = T1rue)
gaze_data = datapicker.dataset.streams.PupilLabs.PupilGaze.data
gaze = gaze_timestamps.join(gaze_data, on='Value') 
gaze = gaze.drop('Value', axis=1)
video_frames = datapicker.dataset.streams.PupilLabs.Counter.DecodedFrames.data
video_frames = video_frames[video_frames.Value !=0]
#eyetracking_correlation = pd.merge_asof(data, space_time, left_index=True, right_index=True)
gaze_coorelation = pd.merge_asof(video_frames, gaze, left_index=True, right_index=True)

gaze_coorelation.to_csv(datapicker._selected_path+r'\gaze_coorelation.csv')
gaze_coorelation